# Pessimistic and Optimistic Bound on the Robustness Radius

In this notebook we try to study the different bounds of the robustness radius given by several methods (lip certificate, adv attacks, CROWN).

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort
from deel.lip.losses import MulticlassHKR, MulticlassKR
from keras.layers import Input, Flatten
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
import keras.ops as K
import matplotlib.pyplot as plt

### Load MNIST Dataset

the dataset has ten classes.

In [3]:
# load data
(x_train, y_train_ord), (x_test, y_test_ord) = mnist.load_data()
# standardize and reshape the data
x_train = np.expand_dims(x_train, -1) / 255
x_test = np.expand_dims(x_test, -1) / 255
# one hot encode the labels
y_train = to_categorical(y_train_ord)
y_test = to_categorical(y_test_ord)


In [4]:
x_train = np.transpose(x_train,(0,3,1,2))
x_test = np.transpose(x_test,(0,3,1,2))

In [5]:
x_train.shape

(60000, 1, 28, 28)

In [6]:
np.max(x_train)

np.float64(1.0)

In [6]:
# model = keras.models.load_model("/home/aws_install/robustess_project/deel-lip/docs/notebooks/demo4_vanilla_fashionMNIST_channelfirst.keras")
# model.compile(
#     # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
#     # note also in the case of lipschitz networks, more robustness require more parameters.
#     loss=MulticlassHKR(alpha=100, min_margin=0.25),
#     optimizer=Adam(1e-4),
#     metrics=["accuracy", MulticlassKR()],)
# model.summary()

In [7]:
vanilla_model = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo0_vanilla_MNIST_channelfirst_False_disj_Neurons.keras")
# HKR (Hinge-Krantorovich-Rubinstein) optimize robustness along with accuracy
vanilla_model.compile(
    # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
    # note also in the case of lipschitz networks, more robustness require more parameters.
    loss=MulticlassHKR(alpha=50, min_margin=0.05),
    optimizer=Adam(1e-3),
    metrics=["accuracy", MulticlassKR()],
)
vanilla_model.summary()

Model: "hkr_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ spectral_conv2d (Conv2D)        │ (None, 16, 28, 28)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ scaled_l2_norm_pooling2d        │ (None, 16, 14, 14)     │             0 │
│ (ScaledL2NormPooling2D)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_conv2d_1 (Conv2D)      │ (None, 16, 14, 14)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ scaled_l2_norm_pooling2d_1      │ (None, 16, 7, 7)       │             0 │
│ (ScaledL2NormPooling2D)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 10)             │           320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,920 (109.06 KB)

 Trainable params: 27,920 (109.06 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# strategy: first
# we select a sample from each class.
images_list = []
labels_list = []
# select only a few element from the test set
# selected = np.random.choice(len(y_test_ord), 500)
sub_y_test_ord = y_test_ord[:400]
sub_x_test = x_test[:400]
# drop misclassified elements
misclassified_mask = K.equal(
    K.argmax(vanilla_model.predict(sub_x_test, verbose=0), axis=-1), sub_y_test_ord
)
sub_x_test = sub_x_test[misclassified_mask.detach().cpu().numpy()]
sub_y_test_ord = sub_y_test_ord[misclassified_mask.detach().cpu().numpy()]

for i in range(10):
    # select the 20 firsts elements of the ith label
    label_mask = sub_y_test_ord == i
    x = sub_x_test[label_mask][:20]
    y = sub_y_test_ord[label_mask][:20]
    # convert it to tensor for use with foolbox
    images = K.convert_to_tensor(x.astype("float32"), dtype="float32")
    labels = K.convert_to_tensor(y, dtype="int64")
    # repeat the input 10 times, one per misclassification target
    for j in range(20):
        images_list.append(images[j])
        labels_list.append(labels[j])
images = K.convert_to_tensor(images_list)
labels = K.convert_to_tensor(labels_list)


In [9]:
labels.shape

torch.Size([200])

images shape = (nombre classes, nbr de points , channels, dim img 1, dim img 2)

### Get the Pessimistic Radius via Lip Constraints (l2 norm)

In [10]:
def compute_certificate(images, model, L=1):    
    values, _ = K.top_k(model(images), k=2)
    certificates = (values[:, 0] - values[:, 1]) / (np.sqrt(2)*L)
    return certificates.detach().cpu().numpy()   

In [11]:
lip_radius = compute_certificate(images, vanilla_model)

In [12]:
lip_radius.shape

(200,)

### Get Optimistic Radius via AutoAttacks

In [13]:
import keras.ops as K
import matplotlib.pyplot as plt
import torchattacks
import torch
import torch.nn as nn
import torchattacks
from robustbench.utils import clean_accuracy

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
def starting_point_dichotomy(idx, images, targets):
    mask_different_classes = targets[idx] != targets
    images_diffferent_classes = images[mask_different_classes]
    return K.amin((images[idx] - images_diffferent_classes).square().sum(dim=(1, 2, 3)).sqrt())

In [15]:
starting_point_dichotomy(1, images, labels)

tensor(8.5971, device='cuda:0')

In [16]:
def single_compute_optimistic_radius_PGD(idx, images, targets, certificates, model, n_iter = 10):
    image = images[idx:idx+1]
    target = targets[idx:idx+1]
    certificate = certificates[idx:idx+1]
    # We use dichotomy algorithm to fine the smallest optimistic radius
    # We start from the closest point with different class
    eps_working = d_up = starting_point_dichotomy(1, images, targets)
    d_low = K.convert_to_tensor(certificate)
    for _ in range(n_iter):
        print(eps_working)
        eps_current = (d_up+d_low)/2
        # atk_van = torchattacks.PGDL2(model, eps=eps_current, alpha=eps_current/5, steps=10, random_start=True)
        atk_van = torchattacks.PGDL2(model, eps=eps_current, alpha=eps_current/5, steps=int((10*eps_current)), random_start=True)
        adv_image = atk_van(image, target)
        # return 0 if the attack doesn't work
        if (K.argmax(model(adv_image), axis=1) == target):
            d_low = eps_current
        else:
            eps_working = d_up = (image - adv_image).square().sum(dim=(1, 2, 3)).sqrt()
    return eps_working

In [17]:
def single_compute_optimistic_radius_AA(idx, images, targets, certificates, model, n_iter = 10):
    image = images[idx:idx+1]
    target = targets[idx:idx+1]
    certificate = certificates[idx:idx+1]
    # We use dichotomy algorithm to fine the smallest optimistic radius
    # We start from the closest point with different class
    eps_working = d_up = starting_point_dichotomy(1, images, targets)
    d_low = d_low = K.convert_to_tensor(certificate)
    for _ in range(n_iter):
        print(eps_working)
        eps_current = (d_up+d_low)/2
        atk = torchattacks.AutoAttack(model, norm='L2', eps=eps_current)
        adv_image = atk(image, target)
        if (K.argmax(model(adv_image), axis=1) == target):
            d_low = eps_current
        else:
            eps_working = d_up = (image - adv_image).square().sum(dim=(1, 2, 3)).sqrt()
    return eps_working

In [20]:
i = 0

In [21]:
single_compute_optimistic_radius_PGD(i, images, labels, lip_radius, vanilla_model, n_iter = 10)

tensor(8.5971, device='cuda:0')
tensor([5.0075], device='cuda:0')
tensor([3.2322], device='cuda:0')
tensor([3.2322], device='cuda:0')
tensor([3.2322], device='cuda:0')
tensor([3.2322], device='cuda:0')
tensor([3.2322], device='cuda:0')
tensor([3.1649], device='cuda:0')
tensor([3.1649], device='cuda:0')
tensor([3.1419], device='cuda:0')


tensor([3.1419], device='cuda:0')

In [22]:
single_compute_optimistic_radius_AA(i, images, labels, lip_radius, vanilla_model, n_iter = 10)

tensor(8.5971, device='cuda:0')
tensor([4.9860], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([3.1579], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([3.1579], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.6946], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.6946], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.6946], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.6037], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.6037], device='cuda:0', grad_fn=<SqrtBackward0>)
tensor([2.6037], device='cuda:0', grad_fn=<SqrtBackward0>)


KeyboardInterrupt: 

### Generating Dataframe

In [ ]:
import pandas as pd

In [ ]:
# penser à tout convertir en numpy
total_points = 200

# Création du DataFrame avec une colonne d'index de 1 à 200
df = pd.DataFrame({
    'Index': np.arange(1, total_points + 1),
    'Label_GT': labels,  
    'Label_Predit': torch.argmax(vanilla_model(images), dim=1),  
    'Constante_Lipschitz': np.ones(total_points), 
    'Epsilon_Robuste': lip_radius,
    'Epsilon_Adv_AA': np.random.rand(total_points),
    'Epsilon_Adv_PGD': np.random.rand(total_points)
})

# Affichage des premières lignes du DataFrame
print(df.head())

   Index  Label_GT  Label_Predit  Constante_Lipschitz  Epsilon_Robuste  \
0      1         6             3             0.350912         0.372034   
1      2         6             9             0.191955         0.136486   
2      3         8             5             0.192445         0.531047   
3      4         7             7             0.536921         0.979334   
4      5         9             6             0.710001         0.329805   

   Epsilon_Adv_AA  Epsilon_Adv_PGD  
0        0.223536         0.717368  
1        0.694969         0.661540  
2        0.109589         0.149326  
3        0.061458         0.299948  
4        0.087260         0.351164  
